In [ ]:
# Explore different layer annotation attributes
print("=== Exploring ev.layers ===")
print(f"Type: {type(ev.layers)}")
if hasattr(ev.layers, 'keys'):
    print(f"Keys: {list(ev.layers.keys())}")
print(f"Content: {ev.layers}")
print(f"ILM layer data: {ev.layers['ILM'].data}")
print()

print("=== Exploring ev._layers ===")
print(f"Type: {type(ev._layers)}")
print(f"Length: {len(ev._layers) if hasattr(ev._layers, '__len__') else 'No length'}")
if hasattr(ev._layers, '__iter__') and len(ev._layers) > 0:
    print(f"First item type: {type(ev._layers[0])}")
    print(f"First item: {ev._layers[0]}")
    if hasattr(ev._layers[0], '__dict__'):
        print(f"First item attributes: {vars(ev._layers[0])}")
print()

# Check if there are other layer-related attributes
print("=== Other potential layer attributes ===")
layer_attrs = [attr for attr in dir(ev) if 'layer' in attr.lower()]
print(f"Layer-related attributes: {layer_attrs}")

# Check annotations
annotation_attrs = [attr for attr in dir(ev) if 'annotation' in attr.lower()]
print(f"Annotation-related attributes: {annotation_attrs}")

In [ ]:
# Function to visualize any B-scan with annotations
def visualize_bscan_with_layers(bscan_idx, figsize=(15, 8))-> None:
    """
    Visualize a B-scan with retinal layer annotations
    
    args:
        bscan_idx: Index of the B-scan to visualize
        figsize: Figure size for the plot
    returns:
        None
    """
    plt.figure(figsize=figsize)
    
    # Display the B-scan image
    plt.imshow(bscans[bscan_idx], cmap='gray', aspect='auto')
    
    selected_layers = ["ILM", "BM", "ELM", "PR1"]
    colors = ['red', 'blue', 'green', 'orange']
    #layer_names = list(layers.keys())
    '''
    for i, layer_name in enumerate(selected_layers):
        layer_data = layers[layer_name].data[bscan_idx]
        
        valid_indices = ~np.isnan(layer_data) # Only plot non- NaN values
        if np.any(valid_indices):
            x_coords = np.where(valid_indices)[0]
            y_coords = layer_data[valid_indices]
            plt.plot(x_coords, y_coords, color=colors[i % len(colors)], 
                    linewidth=2, label=layer_name, alpha=0.8)
    '''
    for i, layer_name in enumerate(selected_layers):
        if layer_name in layers:
            layer_data = layers[layer_name].data[bscan_idx]
            x_coords = np.arange(layer_data.shape[0])
            plt.plot(x_coords, layer_data, color=colors[i % len(colors)],
                    linewidth=2, label=layer_name, alpha=0.8)
    plt.title(f"B-scan {bscan_idx} with Retinal Layer Annotations")
    plt.xlabel("A-scan position (pixel)")
    plt.ylabel("Depth (pixel)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Hdf5 appending logic
    def convert_e2e_file(self, e2e_path: str) -> None:
        """
        Convert a single .e2e file and append it to the HDF5 dataset.
        
        Args:
            e2e_path (str): Path to the .e2e file
        """
        # Extract data from .e2e file
        images, layers, filename = self.extract_e2e_data(e2e_path)
        
        logger.info(f"Adding data from {filename} to HDF5 file: {self.hdf5_path}")
        
        with h5py.File(self.hdf5_path, 'a') as f:  # 'a' mode: create or append
            
            # Handle images dataset
            if 'images' not in f:
                # Create new dataset
                f.create_dataset('images', data=images, 
                               maxshape=(None, *images.shape[1:]),
                               compression='gzip', compression_opts=9)
                logger.info(f"Created new images dataset with shape: {images.shape}")
            else:
                # Append to existing dataset
                existing_images = f['images']
                old_size = existing_images.shape[0]
                new_size = old_size + images.shape[0]
                existing_images.resize((new_size, *existing_images.shape[1:]))
                existing_images[old_size:new_size] = images
                logger.info(f"Appended {images.shape[0]} images. Total: {new_size}")
            
            # Handle layers group
            if 'layers' not in f:
                layers_group = f.create_group('layers')
            else:
                layers_group = f['layers']
            
            # Add each layer
            for layer_name, layer_data in layers.items():
                if layer_name not in layers_group:
                    # Create new layer dataset
                    layers_group.create_dataset(layer_name, data=layer_data,
                                              maxshape=(None, *layer_data.shape[1:]),
                                              compression='gzip', compression_opts=9)
                    logger.info(f"Created new layer dataset '{layer_name}' with shape: {layer_data.shape}")
                else:
                    # Append to existing layer dataset
                    existing_layer = layers_group[layer_name]
                    old_size = existing_layer.shape[0]
                    new_size = old_size + layer_data.shape[0]
                    existing_layer.resize((new_size, *existing_layer.shape[1:]))
                    existing_layer[old_size:new_size] = layer_data
                    logger.info(f"Appended to layer '{layer_name}'. Total: {new_size}")
            
            # Handle filenames dataset
            if 'names' not in f:
                # Create string dataset for filenames
                dt = h5py.string_dtype(encoding='utf-8')
                filenames_array = np.array([filename] * images.shape[0], dtype=dt)
                f.create_dataset('names', data=filenames_array,
                               maxshape=(None,), compression='gzip', compression_opts=9)
                logger.info(f"Created new names dataset")
            else:
                # Append filenames
                existing_names = f['names']
                old_size = existing_names.shape[0]
                new_size = old_size + images.shape[0]
                dt = h5py.string_dtype(encoding='utf-8')
                filenames_array = np.array([filename] * images.shape[0], dtype=dt)
                existing_names.resize((new_size,))
                existing_names[old_size:new_size] = filenames_array
                logger.info(f"Appended filenames. Total: {new_size}")
        
        logger.info(f"Successfully processed: {e2e_path}")